In [1]:
import ScraperFC as sfc
import traceback
import pandas as pd
pd.options.display.max_columns = 50

try:
    scraper = sfc.FBRef()
    df = scraper.scrape_passing(2019, "EPL", normalize=True, player=True)
    scraper.close()
except:
    traceback.print_exc()
    scraper.close()

In [3]:
df.dtypes

Unnamed: 0_level_0   Rk         float64
Unnamed: 1_level_0   Player      object
Unnamed: 2_level_0   Nation      object
Unnamed: 3_level_0   Pos         object
Unnamed: 4_level_0   Squad       object
Unnamed: 5_level_0   Age        float64
Unnamed: 6_level_0   Born       float64
Unnamed: 7_level_0   90s        float64
Total                Cmp        float64
                     Att        float64
                     Cmp%       float64
                     TotDist    float64
                     PrgDist    float64
Short                Cmp        float64
                     Att        float64
                     Cmp%       float64
Medium               Cmp        float64
                     Att        float64
                     Cmp%       float64
Long                 Cmp        float64
                     Att        float64
                     Cmp%       float64
Unnamed: 22_level_0  Ast        float64
Unnamed: 23_level_0  xA         float64
Unnamed: 24_level_0  A-xA       float64
